---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [2]:
# Content of .txt file (uncomment to see)
#!cat university_towns.txt

In [3]:
# Reading txt file into data frame
df_uni = pd.read_table('university_towns.txt', header=None)
df_uni.columns = ['text']
df_uni.shape

(567, 1)

In [4]:
# Strip state from cell
df_uni['State'] = df_uni['text'].apply(lambda x: x.split('[')[0].strip() \
              if x.count('[edit]') > 0 else np.NaN).fillna(method="ffill")

# Strip region from cell
df_uni['RegionName'] = df_uni['text'].apply(lambda x: x.split('(')[0] \
                            .strip() if x.count('(') >= 0 else np.NaN)    
    
df_uni['RegionName'] = df_uni['RegionName'].apply(lambda x: np.NaN \
                                     if x.count('[edit]') > 0 else x)

# Drop initial text column and reset index
df_uni = df_uni.dropna().drop('text', axis=1) \
            .reindex(columns=['State', 'RegionName']).reset_index(drop=True)

In [5]:
df_uni.tail()

,State,RegionName
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater
516,Wyoming,Laramie


In [23]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

# Map inverse dictionary value: key
states_inv = {v: k for k, v in states.items()}

# Map states to acronyms -> Not necessary
df_uni['state_XX'] = df_uni['State'].replace(states_inv)
df_uni.tail()

,State,RegionName,state_XX
512,Wisconsin,River Falls,WI
513,Wisconsin,Stevens Point,WI
514,Wisconsin,Waukesha,WI
515,Wisconsin,Whitewater,WI
516,Wyoming,Laramie,WY


In [24]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from 
    " (" to the end.
    3. Depending on how you read the data, you may need to remove 
    newline character '\n'. '''
    
    # Reading txt file into data frame
    df_uni = pd.read_table('university_towns.txt', header=None)
    df_uni.columns = ['text']
    df_uni.head()
    
    # Strip state from cell
    df_uni['State'] = df_uni['text'].apply(lambda x: x.split('[')[0].strip() \
                if x.count('[edit]') > 0 else np.NaN).fillna(method="ffill")

    # Strip region from cell (aware with the x.count('(') >= 0 )
    df_uni['RegionName'] = df_uni['text'].apply(lambda x: x.split('(')[0] \
                               .strip() if x.count('(') >= 0 else np.NaN)    
    df_uni['RegionName'] = df_uni['RegionName'].apply(lambda x: np.NaN \
                                     if x.count('[edit]') > 0 else x)

    # Drop initial text column and reset index
    df_uni = df_uni.dropna().drop('text', axis=1) \
            .reindex(columns=['State', 'RegionName']).reset_index(drop=True)
    
    # Use this dictionary to map state names to two letter acronyms
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    # Map inverse dictionary value: key
    states_inv = {v: k for k, v in states.items()}
    # Map states to acronyms -> Necessary only for the ttest
    df_uni['State'] = df_uni['State'].replace(states_inv)
    df_uni
    
    return df_uni

In [25]:
get_list_of_university_towns().shape

(517, 2)

#Convert to code to debug
import re
import pandas as pd
import numpy as np
# list of unique states
stateStr = """
Ohio, Kentucky, American Samoa, Nevada, Wyoming
,National, Alabama, Maryland, Alaska, Utah
,Oregon, Montana, Illinois, Tennessee, District of Columbia
,Vermont, Idaho, Arkansas, Maine, Washington
,Hawaii, Wisconsin, Michigan, Indiana, New Jersey
,Arizona, Guam, Mississippi, Puerto Rico, North Carolina
,Texas, South Dakota, Northern Mariana Islands, Iowa, Missouri
,Connecticut, West Virginia, South Carolina, Louisiana, Kansas
,New York, Nebraska, Oklahoma, Florida, California
,Colorado, Pennsylvania, Delaware, New Mexico, Rhode Island
,Minnesota, Virgin Islands, New Hampshire, Massachusetts, Georgia
,North Dakota, Virginia
"""
#list of regionName entries string length
regNmLenStr = """
06,08,12,10,10,04,10,08,09,09,05,06,11,06,12,09,08,10,12,06,
06,06,08,05,09,06,05,06,10,28,06,06,09,06,08,09,10,35,09,15,
13,10,07,21,08,07,07,07,12,06,14,07,08,16,09,10,11,09,10,06,
11,05,06,09,10,12,06,06,11,07,08,13,07,11,05,06,06,07,10,08,
11,08,13,12,06,04,08,10,08,07,12,05,06,09,07,10,16,10,06,12,
08,07,06,06,06,11,14,11,07,06,06,12,08,10,11,06,10,14,04,11,
18,07,07,08,09,06,13,11,12,10,07,12,07,04,08,09,09,13,08,10,
16,09,10,08,06,08,12,07,11,09,07,09,06,12,06,09,07,10,09,10,
09,06,15,05,10,09,11,12,10,10,09,13,06,09,11,06,11,09,13,37,
06,13,06,09,49,07,11,12,09,11,11,07,12,10,06,06,09,04,09,15,
10,12,05,09,08,09,09,07,14,06,07,16,12,09,07,09,06,32,07,08,
08,06,10,36,09,10,09,06,09,11,09,06,10,07,14,08,07,06,10,09,
05,11,07,06,08,07,05,07,07,04,06,05,09,04,25,06,07,08,05,08,
06,05,11,09,07,07,06,13,09,05,16,05,10,09,08,11,06,06,06,10,
09,07,06,07,10,05,08,07,06,08,06,30,09,07,06,11,07,12,08,09,
16,12,11,08,06,04,10,10,15,05,11,11,09,08,06,04,10,10,07,09,
11,08,26,07,13,05,11,03,08,07,06,05,08,13,10,08,08,08,07,07,
09,05,04,11,11,07,06,10,11,03,04,06,06,08,08,06,10,09,05,11,
07,09,06,12,13,09,10,11,08,07,07,08,09,10,08,10,08,56,07,12,
07,16,08,04,10,10,10,10,07,09,08,09,09,10,07,09,09,09,12,14,
10,29,19,07,11,12,13,13,09,10,12,12,12,08,10,07,10,07,07,08,
08,08,09,10,09,11,09,07,09,10,11,11,10,09,09,12,09,06,08,07,
12,09,07,07,06,06,08,06,15,08,06,06,10,10,10,07,05,10,07,11,
09,12,10,12,04,10,05,05,04,14,07,10,09,07,11,10,10,10,11,15,
09,14,12,09,09,07,12,04,10,10,06,10,07,28,06,10,08,09,10,10,
10,13,12,08,10,09,09,07,09,09,07,11,11,13,08,10,07
"""

df = get_list_of_university_towns().copy()

cols = ["State", "RegionName"]

print('Shape test: ', "Passed" if df.shape ==
      (517, 2) else 'Failed')
print('Index test: ',
      "Passed" if df.index.tolist() == list(range(517))
      else 'Failed')

print('Column test: ',
      "Passed" if df.columns.tolist() == cols else 'Failed')
print('\\n test: ',
      "Failed" if any(df[cols[0]].str.contains(
          '\n')) or any(df[cols[1]].str.contains('\n'))
      else 'Passed')
print('Trailing whitespace test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\s+$')) or any(df[cols[1]].str.contains(
              '\s+$'))
      else 'Passed')
print('"(" test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\(')) or any(df[cols[1]].str.contains(
              '\('))
      else 'Passed')
print('"[" test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\[')) or any(df[cols[1]].str.contains(
              '\]'))
      else 'Passed')

states_vlist = [st.strip() for st in stateStr.split(',')]

mismatchedStates = df[~df['State'].isin(
    states_vlist)].loc[:, 'State'].unique()
print('State test: ', "Passed" if len(
    mismatchedStates) == 0 else "Failed")
if len(mismatchedStates) > 0:
    print()
    print('The following states failed the equality test:')
    print()
    print('\n'.join(mismatchedStates))

df['expected_length'] = [int(s.strip())
                         for s in regNmLenStr.split(',')
                         if s.strip().isdigit()]
regDiff = df[df['RegionName'].str.len() != df['expected_length']].loc[
    :, ['RegionName', 'expected_length']]
regDiff['actual_length'] = regDiff['RegionName'].str.len()
print('RegionName test: ', "Passed" if len(regDiff) ==
      0 else ' \nMismatching regionNames\n {}'.format(regDiff))


In [18]:
#Read contents of gdplev.xls
gdp = pd.read_excel('gdplev.xls')
gdp[:20]

,"Current-Dollar and ""Real"" Gross Domestic Product",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,2016-09-29 00:00:00
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Annual,NaN,NaN,NaN,Quarterly,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,(Seasonally adjusted annual rates),NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,GDP in billions of current dollars,GDP in billions of chained 2009 dollars,NaN,NaN,GDP in billions of current dollars,GDP in billions of chained 2009 dollars,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1929,104.6,1056.6,NaN,1947q1,243.1,1934.5,NaN
8,1930,92.2,966.7,NaN,1947q2,246.3,1932.3,NaN
9,1931,77.4,904.8,NaN,1947q3,250.1,1930.3,NaN


In [10]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    # Clean GDP data frame
    gdp = pd.read_excel('gdplev.xls', skiprows=5)
    gdp = gdp.iloc[214:,4:6]
    gdp.columns = [['Quarter', 'GDP']]
    gdp = gdp.reset_index(drop=True)
    gdp['diff'] = gdp['GDP'].diff()
    gdp['diff2'] = [1 if x >= 0 else 0 for x in gdp['diff']]
    
    recession_start_quarters = [] #List of start quarters
    #Look up for the pattern [1]-0-0 to get recession [start]
    for i in range(len(gdp)-2):
        if ((gdp.loc[i, 'diff2'] == 1) & (gdp.loc[i+1, 'diff2'] == 0) &
            (gdp.loc[i+2, 'diff2'] == 0)):
            #Append to list if found:
            recession_start_quarters.append(gdp.loc[i, 'Quarter'])

    return recession_start_quarters[0] #Return just string due to requirements

get_recession_start()

'2008q3'

In [11]:
# Clean GDP data frame
gdp = pd.read_excel('gdplev.xls', skiprows=5)
gdp = gdp.iloc[214:,4:6]
gdp.columns = [['Quarter', 'GDP']]
gdp = gdp.reset_index(drop=True)
gdp['diff'] = gdp['GDP'].diff()
gdp['diff2'] = [1 if x >= 0 else 0 for x in gdp['diff']]
gdp.head()

,Quarter,GDP,diff,diff2
0,2000q1,10031.0,NaN,0
1,2000q2,10278.3,247.3,1
2,2000q3,10357.4,79.1,1
3,2000q4,10472.3,114.9,1
4,2001q1,10508.1,35.8,1


In [12]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    # Clean GDP data frame
    gdp = pd.read_excel('gdplev.xls', skiprows=5)
    gdp = gdp.iloc[214:,4:6]
    gdp.columns = [['Quarter', 'GDP']]
    gdp = gdp.reset_index(drop=True)
    gdp['diff'] = gdp['GDP'].diff()
    gdp['diff2'] = [1 if x >= 0 else 0 for x in gdp['diff']]
    
    recession_end_quarters = [] #List of start quarters
    #Look up for the pattern 0-0-1-[1] to get recession [end]
    for i in range(len(gdp)-3):
        if ((gdp.loc[i, 'diff2'] == 0) & (gdp.loc[i+1, 'diff2'] == 0) &
            (gdp.loc[i+2, 'diff2'] == 1) & (gdp.loc[i+3, 'diff2'] == 1)):
            #Append to list if found:
            recession_end_quarters.append(gdp.loc[i+3, 'Quarter'])

    return recession_end_quarters[0] #Return just string due to requirements

get_recession_end()

'2009q4'

In [13]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    # Clean GDP data frame
    gdp = pd.read_excel('gdplev.xls', skiprows=5)
    gdp = gdp.iloc[214:,4:6]
    gdp.columns = [['Quarter', 'GDP']]
    gdp = gdp.reset_index(drop=True)
    gdp['diff'] = gdp['GDP'].diff()
    gdp['diff2'] = [1 if x >= 0 else 0 for x in gdp['diff']]
    
    #Slice the GDP data frame to the recession period
    gdp = gdp[(gdp['Quarter'] >= get_recession_start()) & 
              (gdp['Quarter'] <= get_recession_end())]
    #Get row where GDP is min()
    bottom = gdp[gdp['GDP'] == gdp['GDP'].min()]
    
    return bottom.iloc[0,0]

get_recession_bottom()

'2009q2'

In [16]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    house_df = pd.read_csv('City_Zhvi_AllHomes.csv')
    house_df.drop(['Metro','CountyName','RegionID','SizeRank'],axis=1,inplace=1)
    #Set multi-index first
    house_df = house_df.set_index(["State", "RegionName"])
        
    #Group columns by quarters and slice from 2000q1 to 2016q3
    #From https://stackoverflow.com/questions/40433717/pandas-how-to-use-groupby-to-group-columns-by-date-in-the-label
    house_df = house_df.groupby(pd.PeriodIndex(house_df.columns, freq='Q'), 
                                axis=1).mean().iloc[:,15:]
    house_df.columns = house_df.columns.strftime('%Yq%q') #COnvert PeriodIndex to String
    return house_df

convert_housing_data_to_quarters().head()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
NY,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
CA,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
IL,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
PA,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
AZ,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0


In [27]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    start = get_recession_start()
    end = get_recession_end()
    bottom = get_recession_bottom()

    house_df = convert_housing_data_to_quarters().copy()
    house_df.sort_index(inplace=True)
    house_df = house_df.loc[:, start:end]
    house_df = house_df.dropna()
    house_df['price_ratio'] = house_df[start].div(house_df[bottom])

    #Create a list (set) with university towns
    unitown_list = get_list_of_university_towns()

    #SLice the house data frame to needed columns
    house_data = house_df[[start, bottom,'price_ratio']]
    house_data = house_data.reset_index()

    #Set intersection
    uni_data = pd.merge(house_data, unitown_list, how='inner', 
                        on=['State', 'RegionName'], indicator=True)

    #Set difference, read on https://stackoverflow.com/questions/42425549/computing-set-difference-in-pandas-between-two-dataframes
    not_uni_data = pd.merge(house_data, unitown_list, how='left', 
                        on=['State', 'RegionName'], indicator=True) \
                        [lambda x: x._merge=='left_only'].drop('_merge',1)

    #print(house_df.shape) #9884
    #print(uni_data.shape) #258
    #print(not_uni_data.shape) #9626 Dimensions OK!

    #ttest
    t,p = ttest_ind(uni_data['price_ratio'], not_uni_data['price_ratio'])

    different = True if p < 0.01 else False
    better = "university town" if uni_data['price_ratio'].mean() \
                < not_uni_data['price_ratio'].mean() else "non-university town"
    #print(uni_data['price_ratio'].mean(), not_uni_data['price_ratio'].mean())
    return (different, p, better)

run_ttest()

(True, 0.0054964273536945918, 'university town')